In [1]:
from typing import *
import sys
import importlib
import inspect
import math
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pickle

In [2]:
from pprint import pprint
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, IBMQ, execute
from qiskit.transpiler.passes import RemoveBarriers
from qiskit.circuit import Parameter
from qiskit.compiler import transpile
import qiskit.ignis.mitigation as mit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_28930/1908178760.py:7: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  import qiskit.ignis.mitigation as mit


In [3]:
shots = 1 << 13
max_iter = 10

In [4]:
raw_hist_list_list = []
meas_fitter_list_list = []
for num_qubits in range(21, 26):
    
    noise_model = NoiseModel()
    for qi in range(num_qubits):
        if qi >= 0:
            read_err = ReadoutError([[0.950, 0.050],[0.050,0.950]])
            noise_model.add_readout_error(read_err, [qi])
            
    raw_hist_list = []
    meas_fitter_list = []
    for i in range(max_iter):
        qc_y = QuantumCircuit(num_qubits)
        qc_y.measure_all()
        job = execute(qc_y, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)
        raw_hist_list.append(job.result().get_counts())
        
        qr = QuantumRegister(num_qubits)
        mit_pattern = [[i] for i in range(num_qubits)]
        meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
        job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=100 * i, seed_simulator=100 * i)
        cal_results = job.result()
        meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
        meas_fitter_list.append(meas_fitter)
    
    raw_hist_list_list.append(raw_hist_list)
    meas_fitter_list_list.append(meas_fitter_list)
    print("size", num_qubits, "finished")

size 21 finished
size 22 finished
size 23 finished
size 24 finished
size 25 finished


In [5]:
with open("pkls/raw_hist_list_list.pkl", "wb") as f:
    pickle.dump(raw_hist_list_list, f)
with open("pkls/meas_fitter_list_list.pkl", "wb") as f:
    pickle.dump(meas_fitter_list_list, f)